In [1]:
import pandas as pd
import numpy as np
import math

from tqdm.notebook import tqdm
import time

In [2]:
path = 'C:/Users/Rusla/Documents/GitHub/Пушкинская карта - стажировка/Sample/'

In [3]:
users = pd.read_csv(path + 'users_full.csv', sep=';', error_bad_lines=False)

In [4]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2879681 entries, 0 to 2879680
Data columns (total 9 columns):
 #   Column              Dtype  
---  ------              -----  
 0   create_date         int64  
 1   user_id             object 
 2   user_birth          object 
 3   create_date.1       object 
 4   user_phone_details  object 
 5   buyer_mobile_phone  object 
 6   region              object 
 7   age                 int64  
 8   user_region         float64
dtypes: float64(1), int64(2), object(6)
memory usage: 197.7+ MB


In [5]:
users.sample(3)

,create_date,user_id,user_birth,create_date.1,user_phone_details,buyer_mobile_phone,region,age,user_region
12473,1634196824,1092d43e4dfa8ae6ffc8007b3d15d3b1,2001-12-26,NaN,NaN,NaN,Краснодарский край,20,23.0
2336985,1637248985,001ea5116e9eea93a4629b76cf1e12d3,2004-11-26,2021-11-18,"iOS, iOS 15.0",82f50274b6402dc885e5b4d69fe6ad8a,NaN,17,NaN
748869,1630522151,0acaf1f0f5fa0969abbc30898863c25d,2000-08-11,2021-11-02,"Android, 11 (SDK 30)",de5ba6173e10b87528b5e046a17f2dd5,Московская область,21,50.0


In [6]:
users = users.astype({"age": "object", "user_region": "object"})

In [7]:
import json 
expl = [json.loads(line) for line in open(path + 'tickets2.json', 'r', encoding='utf-8', errors = 'ignore')]

In [8]:
tickets = pd.DataFrame(expl)

In [9]:
tickets.sample(3)

,payment_date,comment,session_event_id,session_date,session_params,owner,refund_date,buyer_mobile_phone,refund_reason,payment_amount,visitor_first_name,refund_ticket_price,create_date,status,payment_ticket_price,session_organization_id,visit_date
98595,1637733132000,,1448469,1637755200000,Районный дом культуры (Николо-Березовка),9705121040,NaN,fe55306f672397be84f9a3b87e200ed4,None,RUB 500.00,1,None,1637733159574825638,active,RUB 500.00,,NaN
305164,1636359893,,507743,1637163000,"Основной зал (обновленный), 2-ой ярус, 1 ряд, ...",7107520929,NaN,ca42c380ecba0321d84007cd5483b228,None,400.00,,None,1636360206076665821,active,400.00,214,NaN
535087,1637057464,,1370499,1633986000,,7814779935,NaN,0a138a74ef65a36320c69b87333219ed,None,400.00,Лилия,None,1637057466206030290,visited,,26710,1.637057e+09


In [10]:
tickets[tickets['buyer_mobile_phone'].isna()]

,payment_date,comment,session_event_id,session_date,session_params,owner,refund_date,buyer_mobile_phone,refund_reason,payment_amount,visitor_first_name,refund_ticket_price,create_date,status,payment_ticket_price,session_organization_id,visit_date


In [11]:
tickets.visitor_first_name.nunique() # можно было попробовать пол по именм, но имён у нас всего 11 тысяч из 1,2 млн.

11791

# Добавим данные пользователя в дата сет с купленными билетами

In [12]:
tickets = pd.merge(tickets,users,on=['buyer_mobile_phone'], how='left')

In [13]:
tickets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 854330 entries, 0 to 854329
Data columns (total 25 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   payment_date             854330 non-null  int64  
 1   comment                  854330 non-null  object 
 2   session_event_id         854330 non-null  object 
 3   session_date             854330 non-null  int64  
 4   session_params           854330 non-null  object 
 5   owner                    854330 non-null  object 
 6   refund_date              25117 non-null   float64
 7   buyer_mobile_phone       854330 non-null  object 
 8   refund_reason            25117 non-null   object 
 9   payment_amount           854330 non-null  object 
 10  visitor_first_name       854330 non-null  object 
 11  refund_ticket_price      24409 non-null   object 
 12  create_date_x            854330 non-null  int64  
 13  status                   854330 non-null  object 
 14  paym

In [14]:
tickets_2 = tickets[tickets['user_id'].notna() == True]

In [15]:
tickets_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 530576 entries, 1 to 854329
Data columns (total 25 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   payment_date             530576 non-null  int64  
 1   comment                  530576 non-null  object 
 2   session_event_id         530576 non-null  object 
 3   session_date             530576 non-null  int64  
 4   session_params           530576 non-null  object 
 5   owner                    530576 non-null  object 
 6   refund_date              16455 non-null   float64
 7   buyer_mobile_phone       530576 non-null  object 
 8   refund_reason            16455 non-null   object 
 9   payment_amount           530576 non-null  object 
 10  visitor_first_name       530576 non-null  object 
 11  refund_ticket_price      16025 non-null   object 
 12  create_date_x            530576 non-null  int64  
 13  status                   530576 non-null  object 
 14  paym

In [16]:
cliks_add = pd.read_csv(path + 'cliks_add.csv', sep=';', error_bad_lines=False)

In [17]:
cliks_add.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5747389 entries, 0 to 5747388
Data columns (total 17 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   create_time              object 
 1   create_date              object 
 2   user_phone_details       object 
 3   buyer_mobile_phone       object 
 4   user_id                  object 
 5   url                      object 
 6   session_id               float64
 7   session_name             object 
 8   organization_id          float64
 9   organization_name        object 
 10  session_identity         object 
 11  Org_region_number        float64
 12  org_category             object 
 13  age                      float64
 14  user_region              float64
 15  user_phone_details_id    object 
 16  user_phone_details_id_2  int64  
dtypes: float64(5), int64(1), object(11)
memory usage: 745.4+ MB


In [18]:
cliks_add.shape

(5747389, 17)

In [19]:
ticket_users = tickets_2['user_id'].unique()

In [20]:
tickets_2['user_id'].nunique()

329522

In [21]:
len(ticket_users)

329522

In [22]:
cliks_add_users = cliks_add['user_id'].unique()

In [23]:
len(cliks_add_users)

272442

In [24]:
cliks_add['user_id'].nunique()

272442

In [25]:
cliks_add_users[:5]

array(['08014dcd786881f349938c9255e9df36',
       'b2560eaa8bc23dbba5f24aa5075b89ed',
       'fdfe16d86915392aeeca341505cce1c1',
       'cf3f95a25510513018a33990a30f5acb',
       'e2f896fb069e555a5faaf71a3d2b8dec'], dtype=object)

In [26]:
ticket_users[:5]

array(['ef54479d868c93e5b6a6e425fc780954',
       '68f0a5ca2209fedf6d574d74940e55a5',
       '89bc110254468dbf23f51f84385418a0',
       '38d83e61be8f2afd0d335686d685dc56',
       '65835d1c0954fe08db3ffa4e1a5fb75f'], dtype=object)

cliks_and_tickets = cliks_add.copy()

cliks_and_tickets.shape

cliks_and_tickets = pd.DataFrame()

import time
start_time = time.time()

cliks_and_tickets = cliks_add.merge(
    tickets_2.drop_duplicates(subset=['user_id']), left_on='user_id', right_on=['user_id'], how='inner')

print("--- %s seconds ---" % (time.time() - start_time))

cliks_and_tickets.shape

In [27]:
import time
start_time = time.time()

cliks_and_tickets_2 = cliks_add.loc[cliks_add.user_id.isin(tickets_2.user_id.unique())] 

print("--- %s seconds ---" % (time.time() - start_time))

--- 1.501798391342163 seconds ---


In [28]:
cliks_and_tickets_2.shape

(3538545, 17)

In [29]:
cliks_and_tickets_2.user_id.nunique()

137156

In [30]:
#cliks_and_tickets_2.to_csv(path + 'cliks_and_tickets.csv', sep = ';', index = False)

In [31]:
cliks_and_tickets_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3538545 entries, 0 to 5747388
Data columns (total 17 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   create_time              object 
 1   create_date              object 
 2   user_phone_details       object 
 3   buyer_mobile_phone       object 
 4   user_id                  object 
 5   url                      object 
 6   session_id               float64
 7   session_name             object 
 8   organization_id          float64
 9   organization_name        object 
 10  session_identity         object 
 11  Org_region_number        float64
 12  org_category             object 
 13  age                      float64
 14  user_region              float64
 15  user_phone_details_id    object 
 16  user_phone_details_id_2  int64  
dtypes: float64(5), int64(1), object(11)
memory usage: 485.9+ MB


In [32]:
cliks_and_tickets_2 = cliks_and_tickets_2.astype({'session_id': 'object', 'organization_id': 'object', 
                            'Org_region_number': 'object', 'user_region': 'object'}, copy=False)

In [33]:
cliks_and_tickets_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3538545 entries, 0 to 5747388
Data columns (total 17 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   create_time              object 
 1   create_date              object 
 2   user_phone_details       object 
 3   buyer_mobile_phone       object 
 4   user_id                  object 
 5   url                      object 
 6   session_id               object 
 7   session_name             object 
 8   organization_id          object 
 9   organization_name        object 
 10  session_identity         object 
 11  Org_region_number        object 
 12  org_category             object 
 13  age                      float64
 14  user_region              object 
 15  user_phone_details_id    object 
 16  user_phone_details_id_2  int64  
dtypes: float64(1), int64(1), object(15)
memory usage: 485.9+ MB


In [34]:
cliks_and_tickets_2.dropna(subset = ['session_id'], inplace = True)

In [51]:
cliks_and_tickets_2['session_id'].unique()[:5]

array(['717865', '1309020', '1261967', '1247322', '80126'], dtype=object)

In [52]:
cliks_and_tickets_2['user_id'] = cliks_and_tickets_2['user_id'].apply(lambda x: str(x))

In [53]:
cliks_and_tickets_2['session_id'] = cliks_and_tickets_2['session_id'].apply(lambda x: str(x))
cliks_and_tickets_2['session_id'] = cliks_and_tickets_2['session_id'].apply(lambda x: x.replace('.0', ''))

In [54]:
tickets_2['session_event_id'] = tickets_2['session_event_id'].apply(lambda x: str(x))

C:\Users\Rusla\AppData\Local\Temp/ipykernel_16620/2292651450.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tickets_2['session_event_id'] = tickets_2['session_event_id'].apply(lambda x: str(x))


In [55]:
cliks_and_tickets_2['user_session'] = cliks_and_tickets_2['user_id'] + cliks_and_tickets_2['session_id']

In [56]:
tickets_2['user_session'] = tickets_2['user_id'] + tickets_2['session_event_id']

C:\Users\Rusla\AppData\Local\Temp/ipykernel_16620/3420038588.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tickets_2['user_session'] = tickets_2['user_id'] + tickets_2['session_event_id']


In [57]:
cliks_and_tickets_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3526081 entries, 0 to 5747388
Data columns (total 18 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   create_time              object 
 1   create_date              object 
 2   user_phone_details       object 
 3   buyer_mobile_phone       object 
 4   user_id                  object 
 5   url                      object 
 6   session_id               object 
 7   session_name             object 
 8   organization_id          object 
 9   organization_name        object 
 10  session_identity         object 
 11  Org_region_number        object 
 12  org_category             object 
 13  age                      float64
 14  user_region              object 
 15  user_phone_details_id    object 
 16  user_phone_details_id_2  int64  
 17  user_session             object 
dtypes: float64(1), int64(1), object(16)
memory usage: 511.1+ MB


In [58]:
tickets_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 530576 entries, 1 to 854329
Data columns (total 26 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   payment_date             530576 non-null  int64  
 1   comment                  530576 non-null  object 
 2   session_event_id         530576 non-null  object 
 3   session_date             530576 non-null  int64  
 4   session_params           530576 non-null  object 
 5   owner                    530576 non-null  object 
 6   refund_date              16455 non-null   float64
 7   buyer_mobile_phone       530576 non-null  object 
 8   refund_reason            16455 non-null   object 
 9   payment_amount           530576 non-null  object 
 10  visitor_first_name       530576 non-null  object 
 11  refund_ticket_price      16025 non-null   object 
 12  create_date_x            530576 non-null  int64  
 13  status                   530576 non-null  object 
 14  paym

In [59]:
tickets_2['user_session'].unique()[:5]

array(['ef54479d868c93e5b6a6e425fc7809541327993',
       '68f0a5ca2209fedf6d574d74940e55a51315681',
       '89bc110254468dbf23f51f84385418a01255316',
       '38d83e61be8f2afd0d335686d685dc561426221',
       'ef54479d868c93e5b6a6e425fc7809541318121'], dtype=object)

In [60]:
cliks_and_tickets_2['user_session'].unique()[:5]

array(['08014dcd786881f349938c9255e9df36717865',
       'b2560eaa8bc23dbba5f24aa5075b89ed1309020',
       'fdfe16d86915392aeeca341505cce1c11261967',
       'fdfe16d86915392aeeca341505cce1c11247322',
       'fdfe16d86915392aeeca341505cce1c180126'], dtype=object)

In [99]:
cliks_and_tickets_2.drop_duplicates(subset = 'user_session', inplace = True)

In [100]:
cliks_and_tickets_3 = cliks_and_tickets_2.merge(tickets_2[['user_session', 'payment_date', 
                                                           'session_date', 'payment_amount',
                                                          'status', 'payment_ticket_price']], 
                                                on='user_session', how='left')

In [101]:
cliks_and_tickets_3.describe()

,age,user_phone_details_id_2,payment_date,session_date
count,719528.000000,719528.000000,1.486410e+05,1.486410e+05
mean,18.633472,135.289228,1.354053e+11,1.331675e+11
std,2.493590,67.692961,4.482101e+11,4.451008e+11
min,14.000000,15.000000,1.630653e+09,-1.080000e+04
25%,17.000000,110.000000,1.636794e+09,1.636900e+09
50%,19.000000,111.000000,1.637401e+09,1.637950e+09
75%,21.000000,214.000000,1.637927e+09,1.639057e+09
max,23.000000,215.000000,1.638306e+12,1.767092e+12


In [110]:
cliks_and_tickets_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 719528 entries, 0 to 719527
Data columns (total 19 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   create_time              719528 non-null  object 
 1   create_date              719528 non-null  object 
 2   user_id                  719528 non-null  object 
 3   session_id               719528 non-null  object 
 4   session_name             719528 non-null  object 
 5   organization_id          719528 non-null  object 
 6   organization_name        719528 non-null  object 
 7   session_identity         719528 non-null  object 
 8   Org_region_number        717177 non-null  object 
 9   org_category             681177 non-null  object 
 10  age                      719528 non-null  float64
 11  user_region              521446 non-null  object 
 12  user_phone_details_id    719528 non-null  object 
 13  user_phone_details_id_2  719528 non-null  int64  
 14  paym

In [102]:
cliks_and_tickets_3['payment_date'].value_counts()

1.637229e+09    15
1.637317e+09    11
1.637321e+09    10
1.637929e+09    10
1.637928e+09     8
                ..
1.635873e+09     1
1.637774e+09     1
1.636830e+09     1
1.636442e+12     1
1.637158e+12     1
Name: payment_date, Length: 141791, dtype: int64

In [103]:
cliks_and_tickets_3['payment_date'].isna().count()

719528

In [104]:
cliks_and_tickets_3.to_csv(path + 'cliks_and_tickets.csv', sep = ';', index = False)

In [105]:
cliks_and_tickets_3.sample(3)

,create_time,create_date,user_phone_details,buyer_mobile_phone,user_id,url,session_id,session_name,organization_id,organization_name,...,age,user_region,user_phone_details_id,user_phone_details_id_2,user_session,payment_date,session_date,payment_amount,status,payment_ticket_price
487731,2021-11-05 13:33:47,2021-11-05,"Android, 10 (SDK 29)",0e80d03f6785929d9bdb85e10a942250,5b67b8c839454cda9d192ef66cad07ba,https://teatrnabulake.ru/afisha,1256583,Спектакль «Удивительный случай»,12537,ООО «Молодежный театр на Булаке»,...,21.0,NaN,"Android, 10",110,5b67b8c839454cda9d192ef66cad07ba1256583,NaN,NaN,NaN,NaN,NaN
430603,2021-11-15 16:36:29,2021-11-15,"Android, 11 (SDK 30)",7e0fc74f7781060e0be4bbc50c280b38,d41f2ccf544d736ed4752c9780c9ae35,https://tchcenter.ru/afisha-sobytij/#performan...,1042056,Концерт «Музыка Бродвея и Голливуда»,6499,Санкт-Петербургское государственное бюджетное ...,...,18.0,78,"Android, 11",111,d41f2ccf544d736ed4752c9780c9ae351042056,NaN,NaN,NaN,NaN,NaN
399314,2021-11-23 14:34:25,2021-11-23,"iOS, iOS 13.1",5d8823a06a3b378a1b2fae1817cf9e2c,23106b2f805e6fb068c78965d75327e2,https://teatrnur.ru/%D0%B0%D1%84%D0%B8%D1%88%D...,1231187,Спектакль «Шесть невест и один мужчина»,7839,ГБУКИ «Уфимский государственный татарский теат...,...,22.0,2,"iOS, 13",213,23106b2f805e6fb068c78965d75327e21231187,1.637679e+09,1.637762e+09,500.00,active,500.00


In [106]:
cliks_and_tickets_3.drop(['user_phone_details', 'buyer_mobile_phone', 'url', 'user_session'], axis = 1,
                        inplace = True)

In [111]:
cliks_and_tickets_3.sample(3)

,create_time,create_date,user_id,session_id,session_name,organization_id,organization_name,session_identity,Org_region_number,org_category,age,user_region,user_phone_details_id,user_phone_details_id_2,payment_date,session_date,payment_amount,status,payment_ticket_price
27471,2021-11-20 12:09:41,2021-11-20,33a275fb4f4a8a3381507261c0bec0d0,423136,Спектакль «Много шума из ничего»,5973,ОГАУК «Ульяновский драматический театр имени И...,Спектакль,73,NaN,21.0,NaN,"iOS, 15",215,0.000000e+00,NaN,NaN,NaN,NaN
633050,2021-11-13 08:53:06,2021-11-13,892fe2a093f07c756e0fc4fdb3dc2e5c,1247865,"Спектакль «Не жена, а дьявол»",7839,ГБУКИ «Уфимский государственный татарский теат...,Спектакль,2,Театры,20.0,16,"Android, 11",111,0.000000e+00,NaN,NaN,NaN,NaN
187662,2021-11-22 18:20:18,2021-11-22,668e8b5b7e1ca53f74365ff5bb52d4b7,1245359,Спектакль «Мастер и Маргарита»,9958,ФГБУК «Московский Художественный академический...,Спектакль,77,Театры,20.0,26,"iOS, 15",215,1.637606e+09,1.640534e+09,3000,visited,3000
